In [1]:
import osmium
from shapely import wkt
import geopandas
from shapely.geometry import Point, LineString, Polygon

wktfab = osmium.geom.WKTFactory()


class HighwayHandler(osmium.SimpleHandler):
    highways = ["motorway", "trunk", "primary", "secondary", "tertiary"]
    highways_with_level = dict(zip(highways, [1, 2, 3, 4, 5]))

    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.road_names = []
        self.highways = {'id': [],'way_name': [], 'way_geometry': [], 'highway': [], 'way_level': []}

    def get_ways(self, w):
        way_id = w.id
        name = w.tags.get("name") if w.tags.get("name") else "UNKNOWN"
        highway = w.tags.get("highway")
        if highway in HighwayHandler.highways:
            line = wkt.loads(wktfab.create_linestring(w))
            self.append_way_attribute(way_id, name, line, highway, self.get_way_level(highway))

    def way(self, w):
        self.get_ways(w)

    def get_way_level(self, highway_tag: str) -> int:
        return HighwayHandler.highways_with_level.get(highway_tag)

    def append_way_attribute(self, way_id: str, name: str, line: LineString, highway: str, level: int):
        self.highways.get('id').append(way_id)
        self.highways.get('way_name').append(name)
        self.highways.get('way_geometry').append(line)
        self.highways.get('highway').append(highway)
        self.highways.get('way_level').append(level)


In [2]:
h = HighwayHandler()
h.apply_file("taiwan-latest.osm.pbf", locations=True, idx="flex_mem")
result = geopandas.GeoDataFrame(h.highways, geometry="way_geometry")


In [93]:
from shapely.ops import linemerge
ways = result.loc[result.way_name == "重慶北路一段","way_geometry"].values
for i in ways:
    print(i.wkt)
# line1 = ways[0]
# line2 = ways[1]
line_merge = linemerge(ways).wkt
print(line_merge)



LINESTRING (121.5141978 25.0535468, 121.5141678 25.0534056, 121.5140542 25.0528585, 121.5139815 25.052315, 121.5138746 25.051712, 121.513851 25.0515788, 121.513833 25.0515182, 121.5137505 25.051241, 121.5135594 25.0506192, 121.5134602 25.0502835, 121.5132735 25.049556)
LINESTRING (121.5142974 25.0540658, 121.514277 25.0539528, 121.5141978 25.0535468)
LINESTRING (121.5144167 25.0540344, 121.5144376 25.054136)
LINESTRING (121.5132751 25.0473215, 121.5132706 25.047186)
LINESTRING (121.5143339 25.0536428, 121.5143968 25.0539509, 121.5144167 25.0540344)
LINESTRING (121.5132521 25.0493427, 121.5132626 25.0491793, 121.513276 25.0483457, 121.5132796 25.0482732)
LINESTRING (121.5133763 25.0482755, 121.5133771 25.0483412, 121.5133872 25.0491736, 121.513375 25.0493059)
LINESTRING (121.5133588 25.0495362, 121.513559 25.0501188, 121.5136827 25.0504786, 121.5137217 25.0506178, 121.5138527 25.0510841, 121.5139645 25.0515021, 121.5139798 25.0515594)
LINESTRING (121.5132796 25.0482732, 121.5132828 25.0